In [ ]:
from scipy.special import softmax

def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

def multi_logistic_model(X, Y, X_val, Y_val, learning_rate = 0.01, num_iterations = 400 ):
    
    np.random.seed(1)
    lamda = 0.1
    m = Y.shape[0]                          # number of training examples
    n_y = 6                                 # number of classes  
    n_h = 50                                # dimensions of the GloVe vectors 
    
    # Initialize parameters using Xavier initialization
    W = np.random.randn(n_h, n_y) / np.sqrt(n_h)
    b = np.zeros((n_y,))
    theta = np.vstack((W,b))
  
    X = np.hstack((X, np.ones((X.shape[0], 1), dtype=X.dtype)))
    X_val = np.hstack((X_val, np.ones((X_val.shape[0], 1), dtype=X_val.dtype)))
    Y_oh = convert_to_one_hot(Y, C = n_y)

    step_list = [] 
    loss_list = []
    train_accuracy_list = []
    val_accuracy_list = []
     



    #def getLoss(w,x,y,lam):
    #m = x.shape[0] #First we get the number of training examples
    #y_mat = oneHotIt(y) #Next we convert the integer class coding into a one-hot representation
    #scores = np.dot(x,w) #Then we compute raw class scores given our input and current weights
    #prob = softmax(scores) #Next we perform a softmax on these scores to get their probabilities
    #loss = (-1 / m) * np.sum(y_mat * np.log(prob)) + (lam/2)*np.sum(w*w) #We then find the loss of the probabilities
    #grad = (-1 / m) * np.dot(x.T,(y_mat - prob)) + lam*w #And compute the gradient for that loss
    #return loss,grad



    for t in range(num_iterations+1):
      z = np.dot(X,theta)
      a = softmax(z)

      cost = (-1/m)*(np.sum(Y_oh*np.log(a))) #+ (lamda/2)*np.sum(theta*theta)
           
      d_theta = (-1/m)*(np.dot(X.T, (Y_oh - a))) #+ lamda*theta

      theta = theta - learning_rate * d_theta
      
      step_list.append(t) 
      loss_list.append(cost)
          
      train_accuracy_list.append(accuracy(X, Y, theta))
      val_accuracy_list.append(accuracy(X_val, Y_val, theta))

      if t % 100 == 0:
        #learning_rate = learning_rate/(1 + 0.001*t)
        #print('learning rate is ' + str(learning_rate))
        print("Epoch: " + str(t) + " --- cost = " + str(cost))
        print("train_accuracy:" + str(accuracy(X, Y, theta)))
        print("val_accuracy:" + str(accuracy(X_val, Y_val, theta)))
      

      df = pd.DataFrame({
        'step': step_list, 
        'loss': loss_list,
        'train_accuracy' : train_accuracy_list,
        'val_accuracy' : val_accuracy_list
        

    })
    return df, theta
         

def predict(X, theta):
  m = X.shape[0]
  pred = np.zeros((m, 1))
  for j in range(m):                       
        Z = np.dot(X[j],theta)
        A = softmax(Z)
        pred[j] = np.argmax(A)
  return pred


def accuracy(X, Y, theta):
    m = X.shape[0]
    pred = predict(X, theta)
    Accuracy = np.mean((pred[:] == Y.reshape(Y.shape[0],1)[:]))
    return Accuracy
    

def loss_plot(df):
    return df.plot(
            x='step', 
            y='loss',
            xlabel='step',
            ylabel='loss'
        )  
    
def accuracy_plot(df):
    return df.plot(
            x='step', 
            y=['train_accuracy', 'val_accuracy'],
            xlabel='step',
            ylabel='Accuracy'
        )  